## 공공데이터포털

In [1]:
import requests
import pandas as pd

# 1. API 설정
serviceKey = "nEmoa4/43u4WcWznQBC8b4Cd0x6Py+Pz20kqQs6G54f2+fK5gPX084Mv/ss5wMEU1noKUrOhMAxLEooMd69UDw=="
url = "https://api.odcloud.kr/api/ApplyhomeInfoCmpetRtSvc/v1/getAPTLttotPblancCmpet"

# 2. 전체 데이터 조회 (먼저 어떤 데이터가 있는지 확인)
params = {
    'page': 1,
    'perPage': 1000,  # 많은 데이터 조회
    'serviceKey': serviceKey
}

response = requests.get(url, params=params)
data = response.json()

# 3. 데이터 확인
print(f"Status: {response.status_code}")
print(f"응답 키: {data.keys()}")
print(f"데이터 개수: {data.get('totalCount', 0)}개")

# DataFrame 생성
if 'data' in data:
    df = pd.DataFrame(data['data'])
    print(f"\n컬럼 목록:")
    print(df.columns.tolist())
    print(f"\n샘플 데이터:")
    print(df.head())
else:
    print("\n에러 또는 데이터 없음:")
    print(data)

Status: 200
응답 키: dict_keys(['currentCount', 'data', 'matchCount', 'page', 'perPage', 'totalCount'])
데이터 개수: 48116개

컬럼 목록:
['CMPET_RATE', 'HOUSE_MANAGE_NO', 'HOUSE_TY', 'MODEL_NO', 'PBLANC_NO', 'REQ_CNT', 'RESIDE_SECD', 'RESIDE_SENM', 'SUBSCRPT_RANK_CODE', 'SUPLY_HSHLDCO']

샘플 데이터:
  CMPET_RATE HOUSE_MANAGE_NO   HOUSE_TY MODEL_NO   PBLANC_NO REQ_CNT  \
0     (△501)      2025000462  084.9957A       01  2025000462      42   
1     (△496)      2025000462  084.9957A       01  2025000462       5   
2     (△483)      2025000462  084.9957A       01  2025000462      13   
3     (△480)      2025000462  084.9957A       01  2025000462       3   
4     (△132)      2025000462  084.9958B       02  2025000462       6   

  RESIDE_SECD RESIDE_SENM  SUBSCRPT_RANK_CODE  SUPLY_HSHLDCO  
0          01        해당지역                   1            543  
1          02        기타지역                   1            543  
2          01        해당지역                   2            543  
3          02        기타지역      

In [2]:
# 분양정보 API (주소, 날짜 포함)
url_detail = "https://api.odcloud.kr/api/ApplyhomeInfoDetailSvc/v1/getAPTLttotPblancDetail"

response_detail = requests.get(url_detail, params={'page': 1, 'perPage': 1000, 'serviceKey': serviceKey})
data_detail = response_detail.json()

if 'data' in data_detail:
    df_detail = pd.DataFrame(data_detail['data'])
    
    # 경쟁률 + 분양정보 조인
    df_full = df.merge(df_detail, on=['HOUSE_MANAGE_NO', 'PBLANC_NO'], how='inner')
    
    print(f"✓ 총 {len(df_full)}개 데이터")
    print(f"컬럼: {df_full.columns.tolist()}")
    print(df_full[['HSSPLY_ADRES', 'RCRIT_PBLANC_DE', 'CMPET_RATE']].head(10) if 'HOUSE_NM' in df_full.columns else df_full.head())
else:
    print(f"에러: {data_detail}")

✓ 총 1000개 데이터
컬럼: ['CMPET_RATE', 'HOUSE_MANAGE_NO', 'HOUSE_TY', 'MODEL_NO', 'PBLANC_NO', 'REQ_CNT', 'RESIDE_SECD', 'RESIDE_SENM', 'SUBSCRPT_RANK_CODE', 'SUPLY_HSHLDCO', 'BSNS_MBY_NM', 'CNSTRCT_ENTRPS_NM', 'CNTRCT_CNCLS_BGNDE', 'CNTRCT_CNCLS_ENDDE', 'GNRL_RNK1_CRSPAREA_ENDDE', 'GNRL_RNK1_CRSPAREA_RCPTDE', 'GNRL_RNK1_ETC_AREA_ENDDE', 'GNRL_RNK1_ETC_AREA_RCPTDE', 'GNRL_RNK1_ETC_GG_ENDDE', 'GNRL_RNK1_ETC_GG_RCPTDE', 'GNRL_RNK2_CRSPAREA_ENDDE', 'GNRL_RNK2_CRSPAREA_RCPTDE', 'GNRL_RNK2_ETC_AREA_ENDDE', 'GNRL_RNK2_ETC_AREA_RCPTDE', 'GNRL_RNK2_ETC_GG_ENDDE', 'GNRL_RNK2_ETC_GG_RCPTDE', 'HMPG_ADRES', 'HOUSE_DTL_SECD', 'HOUSE_DTL_SECD_NM', 'HOUSE_NM', 'HOUSE_SECD', 'HOUSE_SECD_NM', 'HSSPLY_ADRES', 'HSSPLY_ZIP', 'IMPRMN_BSNS_AT', 'LRSCL_BLDLND_AT', 'MDAT_TRGET_AREA_SECD', 'MDHS_TELNO', 'MVN_PREARNGE_YM', 'NPLN_PRVOPR_PUBLIC_HOUSE_AT', 'NSPRC_NM', 'PARCPRC_ULS_AT', 'PBLANC_URL', 'PRZWNER_PRESNATN_DE', 'PUBLIC_HOUSE_EARTH_AT', 'PUBLIC_HOUSE_SPCLW_APPLC_AT', 'RCEPT_BGNDE', 'RCEPT_ENDDE', 'RCRIT_PBLANC

In [8]:
# 경쟁률 포맷 정리 (정규표현식 없이)
df_full['경쟁률'] = df_full['CMPET_RATE'].str.replace('(', '').str.replace(')', '').str.replace('△', '') + ':1'

# 필요한 컬럼만 선택
df_clean = df_full[[

    'HSSPLY_ADRES',       # 주소
    'RCRIT_PBLANC_DE',    # 공고일
    '경쟁률',              # 깔끔한 경쟁률

]].copy()

# 컬럼명 한글화
df_clean.columns = ['주소', '공고일', '경쟁률']

print(df_clean.head(20))

                                      주소         공고일      경쟁률
0                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    501:1
1                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    496:1
2                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    483:1
3                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    480:1
4                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    132:1
5                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    132:1
6                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    130:1
7                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    130:1
8                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    133:1
9                   부산광역시 사상구 감전동 산1-9일원  2025-10-10    133:1
10                  부산광역시 사상구 감전동 산1-9일원  2025-10-10    131:1
11                  부산광역시 사상구 감전동 산1-9일원  2025-10-10    131:1
12                  부산광역시 사상구 감전동 산1-9일원  2025-10-10   1.05:1
13                  부산광역시 사상구 감전동 산1-9일원  2025-10-10      -:1
14                  부산광역시 사상구 감전동 산1-9일원  2025-10-10      -:1
15      

In [9]:
def get_competition_rate(address):
    """주소로 청약경쟁률 조회"""
    # 키워드 추출
    keywords = [p for p in address.split() if any(p.endswith(s) for s in ['시', '구', '동', '읍', '면'])]
    
    if not keywords:
        return "유효한 주소를 입력해주세요 (예: 서울 강남구)"
    
    # OR 조건으로 한번에 검색 (키워드 중 하나라도 포함)
    pattern = '|'.join(keywords)
    result = df_clean[df_clean['주소'].str.contains(pattern, na=False)]
    
    return result.sort_values('공고일', ascending=False).reset_index(drop=True) if len(result) > 0 else f"'{address}'에 해당하는 청약 데이터가 없습니다."

# 사용 예시
print(get_competition_rate("경기도 의정부시 용현동 267-8번지"))
print("\n" + "="*60 + "\n")
print(get_competition_rate("경기도 양주시 덕계동 152번지 일원"))
print("\n" + "="*60 + "\n")
print(get_competition_rate("전북 전주시 완산구 서신동 비사벌 아파트 101동 702호"))

                         주소         공고일     경쟁률
0   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    20:1
1   경기도 의정부시 용현동 267-8번지 일원  2025-08-25  3.35:1
2   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    90:1
3   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    86:1
4   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    86:1
5   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    86:1
6   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    48:1
7   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    47:1
8   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    47:1
9   경기도 의정부시 용현동 267-8번지 일원  2025-08-25    46:1
10  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    97:1
11  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    94:1
12  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    94:1
13  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    91:1
14  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    14:1
15  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    13:1
16  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    13:1
17  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    98:1
18  경기도 의정부시 용현동 267-8번지 일원  2025-08-25    99:1
19  경기도 의정부시 용현동 267-8번지 일원  2025-08-25 

## 사용 방법

1. **첫 번째 셀 실행**: API로 전체 청약 데이터 조회
2. **두 번째 셀 실행**: 주소로 청약경쟁률 검색

---

### 주소 예시
- `"서울특별시 강남구 역삼동"`
- `"경기도 성남시 분당구 정자동"`
- `"부산광역시 해운대구 우동"`
- `"전북 전주시 완산구 서신동 비사벌 아파트 101동 702호"`